In [4]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import depth_pro
matplotlib.use("qt5agg")  # For Qt-based interactivity
#%matplotlib


/home/exjobb/anaconda3/envs/depth-pro/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
# Load model and preprocessing transform
model, transform = depth_pro.create_model_and_transforms()
model.eval()

# Load and preprocess an image.
image_path = "../../../test_images/forest_with_road.jpeg"
image, _, f_px = depth_pro.load_rgb(image_path)
image = transform(image)

# Run inference.
prediction = model.infer(image, f_px=f_px)
depth = prediction["depth"]  # Depth in [m].
focallength_px = prediction["focallength_px"]  # Focal length in pixels.

print("done")

/home/exjobb/exjobb/eval_models/depth_pro/ml-depth-pro/src/depth_pro/depth_pro.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(config.checkpoin

done


In [38]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

sequence_containing_x_vals = list(range(0, 100))
sequence_containing_y_vals = list(range(0, 100))
sequence_containing_z_vals = list(range(0, 100))

random.shuffle(sequence_containing_x_vals)
random.shuffle(sequence_containing_y_vals)
random.shuffle(sequence_containing_z_vals)

ax.scatter(sequence_containing_x_vals, sequence_containing_y_vals, sequence_containing_z_vals)
plt.show()

In [39]:
print(depth.shape)
print(depth.shape[0]*depth.shape[1])
print(focallength_px)

torch.Size([142, 355])
50410
tensor(355.9343)


In [40]:
# Image dimensions
image_height, image_width = depth.shape

# Generate a meshgrid for pixel coordinates
u, v = np.meshgrid(np.arange(image_width), np.arange(image_height))

# Normalize pixel coordinates to camera space
x_normalized = (u - image_width / 2) / focallength_px
y_normalized = (v - image_height / 2) / focallength_px

# Back-project to 3D using depth
z = depth
x = x_normalized * z
y = y_normalized * z

# Normalize vectors to unit length
norm = np.sqrt(x**2 + y**2 + z**2)
x_unit = x / norm
y_unit = y / norm
z_unit = z / norm

# Flatten arrays for 3D scatter plot
x_unit_flat = x_unit.flatten()
y_unit_flat = y_unit.flatten()
z_unit_flat = z_unit.flatten()

# Plot the points
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')
ax.scatter(x_unit_flat, y_unit_flat, z_unit_flat, s=1, alpha=0.5)
ax.set_title("Projection of Unit Vectors")
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
#plt.show()

Text(0.5, 0, 'Z')

In [41]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def draw_camera(ax, position, direction, focal_length, up_vector=np.array([0, 1, 0]), size=0.5):
    """
    Draws a 3D representation of a camera in a Matplotlib plot.

    Parameters:
    - ax: Matplotlib 3D axis to draw the camera on.
    - position: np.array, the 3D position of the camera (x, y, z).
    - direction: np.array, the direction the camera is pointing.
    - focal_length: float, the focal length controlling the size of the frustum.
    - up_vector: np.array, the "up" direction for the camera (default is [0, 1, 0]).
    - size: float, a scaling factor for the size of the camera representation.
    """
    # Normalize the direction and up vector
    direction = direction / np.linalg.norm(direction)
    up_vector = up_vector / np.linalg.norm(up_vector)
    
    # Right vector (cross product of direction and up)
    right_vector = np.cross(direction, up_vector)
    right_vector /= np.linalg.norm(right_vector)
    
    # Recompute the up vector to ensure orthogonality
    up_vector = np.cross(right_vector, direction)
    
    # Camera corners in 3D space
    half_width = size
    half_height = size * 0.75
    camera_base = position + direction * focal_length  # Camera base center (image plane)
    
    top_left = camera_base - half_width * right_vector + half_height * up_vector
    top_right = camera_base + half_width * right_vector + half_height * up_vector
    bottom_left = camera_base - half_width * right_vector - half_height * up_vector
    bottom_right = camera_base + half_width * right_vector - half_height * up_vector
    
    # Draw the frustum (lines from camera position to image plane corners)
    for corner in [top_left, top_right, bottom_left, bottom_right]:
        ax.plot(
            [position[0], corner[0]],
            [position[1], corner[1]],
            [position[2], corner[2]],
            'k-'
        )
    
    # Draw the image plane (rectangle at the base)
    ax.plot(
        [top_left[0], top_right[0], bottom_right[0], bottom_left[0], top_left[0]],
        [top_left[1], top_right[1], bottom_right[1], bottom_left[1], top_left[1]],
        [top_left[2], top_right[2], bottom_right[2], bottom_left[2], top_left[2]],
        'b-'
    )
    
    # Draw the camera body (a small sphere at the position)
    ax.scatter(position[0], position[1], position[2], color='red', s=50, label='Camera')
    ax.text(position[0], position[1], position[2], 'Camera', color='red')

# Example Usage
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

# Example camera parameters
camera_position = np.array([0, 0, 0])
camera_direction = np.array([1, 0, 0])  # Points along the Z-axis
focal_length = 1.5

draw_camera(ax, camera_position, camera_direction, focal_length)

# Set axis labels and limits for better visualization
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
ax.set_zlim(-2, 2)

plt.legend()
plt.show()


In [19]:

def draw_camera(
    ax,
    direction,
    focal_length,
    center=np.array([0.0, 0.0, 0.0]),
    up=np.array([0.0, 1.0, 0.0]),
    sensor_width=1.0,
    sensor_height=1.0,
    color='blue'
):
    """
    Draws a simple 3D representation of a camera frustum on a Matplotlib 3D axis.

    Parameters
    ----------
    ax : mpl_toolkits.mplot3d.axes3d.Axes3D
        A 3D axis from matplotlib for plotting.
    direction : array-like of shape (3,)
        The camera's viewing (optical) axis vector in 3D space.
    focal_length : float
        The distance from the camera center to the image plane along `direction`.
    center : array-like of shape (3,), optional
        The 3D coordinates of the camera center/origin.
    up : array-like of shape (3,), optional
        A vector indicating the 'up' direction for the camera.
    sensor_width : float, optional
        The width of the image plane (in some consistent 3D unit).
    sensor_height : float, optional
        The height of the image plane (in some consistent 3D unit).
    color : str, optional
        Color for the camera lines.
    """

    # Convert inputs to numpy arrays
    center = np.array(center, dtype=float).reshape(3)
    direction = np.array(direction, dtype=float).reshape(3)
    up = np.array(up, dtype=float).reshape(3)

    # Normalize the optical axis (z-axis of camera)
    z_cam = direction / np.linalg.norm(direction)

    # Create an orthogonal coordinate system: find right (x_cam) and up (y_cam).
    # We assume 'up' is not perfectly aligned with 'direction'.
    x_cam = np.cross(z_cam, up)
    x_cam /= np.linalg.norm(x_cam)

    y_cam = np.cross(x_cam, z_cam)
    y_cam /= np.linalg.norm(y_cam)

    # Ensure the basis vectors are orthogonal
    assert np.isclose(np.dot(x_cam, y_cam), 0), "x_cam and y_cam are not orthogonal."
    assert np.isclose(np.dot(y_cam, z_cam), 0), "y_cam and z_cam are not orthogonal."
    assert np.isclose(np.dot(z_cam, x_cam), 0), "z_cam and x_cam are not orthogonal."

    # Center of the image plane is focal_length away along the optical axis
    plane_center = center + z_cam * focal_length

    # Half sizes of the sensor
    half_w = sensor_width / 2.0
    half_h = sensor_height / 2.0

    # Image-plane corners in camera space
    # corners at (±half_w, ±half_h) around plane_center
    corners = []
    for dx in [-half_w, half_w]:
        for dy in [-half_h, half_h]:
            corner = plane_center + dx * x_cam + dy * y_cam
            corners.append(corner)

    # Plot lines from the camera center to each corner
    for corner in corners:
        xs = [center[0], corner[0]]
        ys = [center[1], corner[1]]
        zs = [center[2], corner[2]]
        ax.plot(xs, ys, zs, color=color, linewidth=1.0)

    # Plot the rectangle (image plane) by connecting corners in pairs
    # We'll connect corners in the order: 
    # 0 -> 1, 1 -> 3, 3 -> 2, 2 -> 0  (assuming the enumeration below)
    # corners: [(-w, -h), (w, -h), (-w, h), (w, h)]
    corner_idx_pairs = [(0,1), (1,3), (3,2), (2,0)]
    for i1, i2 in corner_idx_pairs:
        c1 = corners[i1]
        c2 = corners[i2]
        ax.plot([c1[0], c2[0]], [c1[1], c2[1]], [c1[2], c2[2]], color=color, linewidth=1.5)

    # Optionally, mark the camera center
    ax.scatter(center[0], center[1], center[2], color=color, s=40, marker='o')
    
    # A small label or arrow to show direction
    # We'll draw a short arrow along the optical axis for clarity
    arrow_scale = focal_length * 0.5  # short arrow in front of camera
    arrow_end = center + z_cam * arrow_scale
    ax.plot([center[0], arrow_end[0]],
            [center[1], arrow_end[1]],
            [center[2], arrow_end[2]],
            color='red', linewidth=2)
    
    # Label the arrow as "direction"
    ax.text(arrow_end[0], arrow_end[1], arrow_end[2], "Optical Axis", color='red')
    
    # Some optional axis labeling
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

In [ ]:
def draw_point(ax):
    pass

In [7]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

# Draw two cameras for demonstration:
# draw_camera(ax, direction=[0, 0, 1], focal_length=2.0, color='blue')
direction=[1, 1, 1]
focal_length=1.0
center=[-2, -2, -1]
draw_camera(ax, direction, focal_length,
            center, up=[0, 0, 1], color='green')



# Adjust the viewing angle and show the plot
ax.view_init(elev=20., azim=30.)
plt.title("3D Camera Visualization")
plt.show()

In [28]:
import numpy as np
import matplotlib.pyplot as plt

###############################################################################
# 1) Helper: Build a rotation + translation that orients the camera.
#    direction: which way the camera is pointing (camera +Z axis in world).
#    up       : which way is "up" (camera +Y axis in world).
#    center   : the camera origin in world coordinates (Tx, Ty, Tz).
###############################################################################
def get_camera_transform(direction, up, center):
    """
    Returns a (3x3, 1x3) tuple: (R, t), 
    so that a camera-space point X_cam is mapped to world space as:
        X_world = R @ X_cam + t
    """
    direction = np.array(direction, dtype=float)
    up        = np.array(up,        dtype=float)
    center    = np.array(center,    dtype=float)

    # Normalize the forward (Z) vector
    forward = direction / np.linalg.norm(direction)

    # Enforce that up is orthogonal to forward
    # (This may not be strictly necessary if up is already orthogonal)
    up_proj = up - np.dot(up, forward) * forward
    up_norm = up_proj / np.linalg.norm(up_proj)

    # Right axis (camera +X) via cross product
    right = np.cross(up_norm, forward)
    right /= np.linalg.norm(right)

    # Build rotation matrix from camera coords to world coords
    # Camera coordinate system: 
    #   +X = right, +Y = up_norm, +Z = forward
    R = np.column_stack((right, up_norm, forward))

    # Translation is just the camera center in world coords
    t = center.reshape((3,1))

    return R, t

###############################################################################
# 2) Draw a single 3D point that the camera "sees" using approach #1 (Z-depth).
###############################################################################
def draw_point(ax, R, t, 
               u=50, v=50,  # pixel coords in some arbitrary image
               cx=0, cy=0, # principal point
               f=100,      # focal length in pixels
               depth=5.0   # Z-distance along camera's optical axis
               ):
    """
    1) Back-project pixel (u,v) at 'depth' using a pinhole camera 
       with focal length f and principal point (cx, cy).
    2) Transform that camera-space point into world-space with (R,t).
    3) Draw on ax.
    """

    # --- A) Compute camera-space coordinates (X_cam, Y_cam, Z_cam)
    X_cam = ((u - cx) / f) * depth
    Y_cam = ((v - cy) / f) * depth
    Z_cam = depth

    # Put into a vector
    point_cam = np.array([[X_cam], [Y_cam], [Z_cam]])  # shape (3,1)

    # --- B) Convert to world coordinates using extrinsic:
    #     X_world = R @ X_cam + t
    point_world = R @ point_cam + t

    # --- C) Plot the point
    ax.scatter(point_world[0,0], 
               point_world[1,0], 
               point_world[2,0],
               c='red', s=50, marker='.')

###############################################################################
# 3) Example usage
###############################################################################
if __name__ == "__main__":
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Suppose we have a camera with:
    direction   = [1, 1, 1]   # it points roughly along (1,1,1)
    up          = [0, 0, 1]   # 'up' is global Z
    center      = [-2, -2, -1]# camera location in world coords
    focal_length= 1.0         # we'll let 'draw_camera' handle this for visualization

    # We'll call some external function 'draw_camera' you might have
    # that draws a frustum or just an axis triad for the camera.
    # For demonstration we assume you have that function:
    # draw_camera(ax, direction, focal_length, center, up=up, color='green')

    # Now get the transform from camera coords -> world coords
    R, t = get_camera_transform(direction, up, center)

    # Draw a single example 3D point the camera sees
    for x in range(10):
        for y in range(10):
            draw_point(ax, R, t, 
                    u=(x-5)*10, v=(y-5)*10,   # example pixel coordinate
                    cx=0, cy=0,   # principal point
                    f=100,        # focal length in pixels
                    depth=2.0)    # the Z-distance for the point

    #direction=[1, 1, 1]
    #focal_length=1.0
    #center=[-2, -2, -1]
    draw_camera(ax, direction, focal_length,
                center, up=[0, 0, 1], color='green')

    # Set fixed ranges for the axes
    ax.set_xlim(-2, 2)  # Set X-axis range from -10 to 10
    ax.set_ylim(-2, 2)  # Set Y-axis range from -10 to 10
    ax.set_zlim(-2, 2)  # Set Z-axis range from -10 to 10

    # Tweak the view and show
    ax.view_init(elev=20., azim=30.)
    plt.title("3D Camera Visualization with Example Point")
    plt.legend()
    plt.show()


/tmp/ipykernel_75525/2109311165.py:120: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
